### 1. Import Libraries

In [120]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import os

### 2. Data Obtention

In [121]:
tickers = yf.Tickers('AES BTU SCCO UNP')    
hist = tickers.history(start='2021-01-01',end='2024-12-31')
print('Data:')
print(hist)

[*********************100%***********************]  4 of 4 completed

Data:
Price           Close                                   Dividends            \
Ticker            AES        BTU       SCCO         UNP       AES  BTU SCCO   
Date                                                                          
2021-01-04  20.197662   2.860808  54.866444  185.846634       0.0  0.0  0.0   
2021-01-05  20.875547   3.337609  55.592339  188.126175       0.0  0.0  0.0   
2021-01-06  21.709875   3.201380  56.236668  192.062607       0.0  0.0  0.0   
2021-01-07  21.492601   3.181919  58.055492  194.735733       0.0  0.0  0.0   
2021-01-08  22.092274   3.123535  58.267567  200.320023       0.0  0.0  0.0   
...               ...        ...        ...         ...       ...  ...  ...   
2024-12-23  12.856047  20.203943  92.899857  227.130005       0.0  0.0  0.0   
2024-12-24  12.816671  19.955126  93.753693  229.750000       0.0  0.0  0.0   
2024-12-26  12.639482  19.716261  93.922478  230.229996       0.0  0.0  0.0   
2024-12-27  12.550888  19.865551  92.542435  2

In [122]:
adj_close = hist['Close']
#adj_close = hist['Close'].fillna(method='ffill')
#adj_close = adj_close.fillna(method='bfill')  
print('Adj Close:')
print(adj_close)

Adj Close:
Ticker            AES        BTU       SCCO         UNP
Date                                                   
2021-01-04  20.197662   2.860808  54.866444  185.846634
2021-01-05  20.875547   3.337609  55.592339  188.126175
2021-01-06  21.709875   3.201380  56.236668  192.062607
2021-01-07  21.492601   3.181919  58.055492  194.735733
2021-01-08  22.092274   3.123535  58.267567  200.320023
...               ...        ...        ...         ...
2024-12-23  12.856047  20.203943  92.899857  227.130005
2024-12-24  12.816671  19.955126  93.753693  229.750000
2024-12-26  12.639482  19.716261  93.922478  230.229996
2024-12-27  12.550888  19.865551  92.542435  229.929993
2024-12-30  12.452449  20.681673  90.795036  227.789993

[1004 rows x 4 columns]


In [123]:
if os.path.exists('data4.xlsx'):
    os.remove('data4.xlsx')
adj_close.to_excel('data4.xlsx')

### 3. Portfolio Analysis

In [124]:
adj_close_values = adj_close.values
print('Adj Close values:')
print(np.round(adj_close_values,4))
print(adj_close_values.shape)

Adj Close values:
[[ 20.1977   2.8608  54.8664 185.8466]
 [ 20.8755   3.3376  55.5923 188.1262]
 [ 21.7099   3.2014  56.2367 192.0626]
 ...
 [ 12.6395  19.7163  93.9225 230.23  ]
 [ 12.5509  19.8656  92.5424 229.93  ]
 [ 12.4524  20.6817  90.795  227.79  ]]
(1004, 4)


In [125]:
R = np.log(adj_close_values[1:] / adj_close_values[:-1])
print('R:')
print(np.round(R*100, 4), '%')

R:
[[ 3.3012 15.4151  1.3143  1.2191]
 [ 3.9189 -4.1673  1.1524  2.0709]
 [-1.0058 -0.6098  3.183   1.3822]
 ...
 [-1.3921 -1.2042  0.1799  0.2087]
 [-0.7034  0.7543 -1.4802 -0.1304]
 [-0.7874  4.0261 -1.9063 -0.9351]] %


In [126]:
RE = np.mean(R, axis=0)*252
RI = np.std(R, axis=0)*np.sqrt(252)
print('RE:', np.round(RE*100, 4), '%')
print('RI:', np.round(RI*100, 4), '%')

RE: [-12.1515  49.7001  12.6553   5.1129] %
RI: [34.3522 76.8036 37.2596 22.4791] %


In [127]:
Sharpes = RE / RI
print('Sharpes:', np.round(Sharpes, 4))

Sharpes: [-0.3537  0.6471  0.3397  0.2275]


In [128]:
S = np.cov(R, rowvar=False)
print('S:')
print(S*100, '%')

S:
[[0.04687514 0.01097494 0.01671421 0.01098244]
 [0.01097494 0.23431288 0.03698926 0.01323317]
 [0.01671421 0.03698926 0.05514539 0.01017095]
 [0.01098244 0.01323317 0.01017095 0.02007201]] %


In [129]:
corr = np.corrcoef(R, rowvar=False)
print('Correlation:')
print(corr)

Correlation:
[[1.         0.10472075 0.32874567 0.35804062]
 [0.10472075 1.         0.32540402 0.1929614 ]
 [0.32874567 0.32540402 1.         0.30571124]
 [0.35804062 0.1929614  0.30571124 1.        ]]


In [130]:
n_assets = R.shape[1]
weights = np.ones(n_assets) / n_assets
print('Weights:', np.round(weights, 4))
print('Sum of weights:', np.sum(weights))

Weights: [0.25 0.25 0.25 0.25]
Sum of weights: 1.0


In [131]:
def rep(w,r):
    return w @ np.transpose(r)

def varp(w,s):
    return w @ s @ np.transpose(w)

In [132]:
ReP = rep(weights, RE)
varP = varp(weights, S)
RiP = np.sqrt(varP)
SharpeP = ReP / RiP
print('Portafolio pre-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio pre-optimización:
ReP: 13.8292 %
varP: 0.0003
RiP: 1.8617 %
SharpeP: 7.4284


### 4. Portfolio Optimization

In [133]:
def constr(w):
    return np.sum(w) - 1

num_assets = 4
bounds = [(0, None)] * num_assets 

constraints = [{'type': 'eq', 'fun': constr}]

res = minimize(
    fun=lambda w: varp(w, S),
    x0=weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12}
)

print('Pesos optimizados:')
print(np.round(res.x*100, 4), '%')


Pesos optimizados:
[15.5071  0.5759 13.2719 70.6451] %


In [134]:
ReP = rep(res.x, RE)
varP = varp(res.x, S)   
RiP = np.sqrt(varP)
SharpeP = ReP / RiP
print('Portafolio post-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio post-optimización:
ReP: 3.6935 %
varP: 0.0002
RiP: 1.3156 %
SharpeP: 2.8074
